In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install pycaret
from pycaret.regression import *

In [ ]:
# Read the data
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', index_col='row_id')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col='row_id')
submission = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv")

In [ ]:
def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
train.head()

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

In [ ]:
train['time']= pd.to_datetime(train['time'])
test['time']= pd.to_datetime(test['time'])

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
# Feature Engineering
for df in [train, test]:
    df['hour'] = df['time'].dt.hour
    df['minute'] = df['time'].dt.minute

In [ ]:
median=train.groupby(['hour','minute','x','y','direction']).median()
test=test.merge(median,how='left',on=['x','y','direction','hour','minute'])

> PyCaret - AutoMl

In [ ]:
model = setup(data = train, 
          target = 'congestion', 
          use_gpu = True,
          n_jobs = -1,
          silent = True,
          fold_shuffle = True,
          fold = 5
         )

best_model = compare_models(exclude=['et'])